# Cadenas de Markov 
También son llamados como procesos de Markov.
## Definición
Es un proceso estocástico con la propiedad de que para cualquier conjunto sucesivo de 
$n$ tiempos tales que $t_{1} < t_{2} < \dotsc < t_{n}$ se tiene que:
$$
    P_{1 | n - 1}\left(y_{n} , t_{n} | y_{1}, t_{1}; \dotsc ; y_{n - 1}, t_{n-1}\right) = P_{1 | 1}\left( y_n, t_n | y_{n - 1}, t_{n - 1}\right)
$$
Esto es que la densidad de probabilidad condicionada a $t_{n}$ dada el valor $y_{n-1}$ a $t_{n-1}$ está unívocamente determinada y no está afectada por lo que ocurre a tiempos anteriores. A $P_{1 | 1}$ se le conoce como la **probabilidad de transición**.
## Definición (no tan formal).
Una cadena o proceso de **Markov** es un proceso evolutivo que consiste de un número finito de estados en cual la probabilidad de que ocurra un evento depende solamente del evento inmediatamente anterior con unas probabilidades que están fijas.


# Ejemplo
## Mercado de valores
![Cadena de Markov](imgs/Finance_Markov_chain_example_state_space.jpg)

Los estados representan si un mercado bursátil (hipotético) muestra una tendencia alcista, bajista o de estancamiento durante una semana determinada. Según la gráfica dirigida anterior, sí pasamos una semana con tendencia alcista, entonces hay probabilidad de $90\%$ que la siguiente semana sea con tendencia alcista, $7.5\%$ de que sea una semana con tendencia a la baja y $2.5\%$ de que la siguiente semana se estanque.
Notemos que la gráfica anterior cumple lo siguiente:
* **La pesos de todas las aristas que salen del estado $S$, suman $1$.**

*Lo anterior se sigue de los axiomas de probabilidad.*

# Matriz de transición. 
$$
    A =
    \begin{bmatrix}
        0.9  & 0.075 & 0.025 \\
        0.15 & 0.8   & 0.05 \\
        0.25 & 0.25  & 0.5
    \end{bmatrix}
$$

In [2]:
import numpy as np

matrix_transition = np.matrix([[0.9, 0.075, 0.025], [0.15, 0.8, 0.05], [0.25, 0.25, 0.5]]) # Bull Market Bear Market Stagnant market

def random_walk(num_iter):
    init = np.random.randint(0, 3)
    bull_market = 0
    bear_market = 0
    stagnant_market = 0
    for i in range(num_iter):
        r = np.random.rand()
        if init == 0:
            if r < 0.9:
                bull_market += 1
            elif r >= 0.9 and r < 0.975:
                init = 1
                bear_market += 1
            else:
                init = 2
                stagnant_market += 1
        elif init == 1:
            if r < 0.15:
                init = 0
                bull_market += 1
            elif r >= 0.15 and r < 0.95:
                bear_market += 1
            else: 
                init = 2
                stagnant_market += 1
        else:
            if r < 0.25:
                init = 0
                bull_market += 1
            elif r >= 0.25 and r < 0.5:
                init = 1
                bear_market += 1
            else:
                stagnant_market += 1
    return (bull_market, bear_market, stagnant_market)
    
markets = random_walk(1000000)
total = markets[0] + markets[1] + markets[2]
print("Probabilidad de un mercado alcista:   " + str(markets[0] / total))
print("Probabilidad de un mercado bajista:   " + str(markets[1] / total))
print("Probabilidad de un mercado estancado: " + str(markets[2] / total))

Probabilidad de un mercado alcista:   0.624963
Probabilidad de un mercado bajista:   0.312889
Probabilidad de un mercado estancado: 0.062148


# Distribución estacionaria
Sea $p^{(t)}$ la probabilidad de la distribución después de $t$ pasos de un *random walk*. Definimos la distribución de probabilidad $a^{(t)}$ como sigue:
$$
    a^{(t)} = \frac{1}{t}\left(p^{(0)} + p^{(1)} + \dotsc + p^{(t - 1)}\right)
$$
Es decir, ésta distribución de probabilidad **no cambia con el tiempo** en la cadena de Markov.

# Teorema Fundamental de las cadenas de Markov
Para toda cadena de Markov conectada existe un único vector $\pi$ de probabilidad que satisface:
$$
    \pi P = \pi
$$
donde $P$ es la matriz de transición asociada a la cadena de Markov. Más aún, para cualquier distribución inicial, $\lim_{t \to \infty} a^{t}$ existe y es igual a $\pi$

In [3]:
bull_market = np.array([1, 0, 0]) # Supongamos que en el tiempo t = 0 es una semana con tendencia alcista.

def find_distribution(matrix_transition, init):
    pi = np.dot(init, matrix_transition)
    while (pi != init).any():
        init = pi
        pi = np.dot(init, matrix_transition)
    return pi

print(find_distribution(matrix_transition, bull_market))

[[0.625  0.3125 0.0625]]


Notemos que nuestra aproximación ejecutando *random walk*  es bastante buena y que de hecho cada que el número de iteraciones es más grande se va aproximando más al método con los vectores.

Notemos que como el vector $\pi$ denota una distribución de probabilidad entonces, el total de la suma de sus componentes es $1$.

Así por lo que obtuvimos anteriormente concluimos lo siguiente:
* El $62.5 \%$ de las semanas serán a la alza.
* El $31.25 \%$ de las semanas se estancarán.
* El $6.25 \%$ de las semanas serán a la baja.

# Irreducibilidad y Recurrencia
![Cadena de Markov](imgs/cadena_de_markov_1.png)

Veamos el estado $0$.
Notemos que sí hacemos varios *random walk* sobre la cadena de Markov anterior iniciando en el estado $0$, habrá caminos que no saldrán del estado $0$, es decir, se quedarán en un **loop** sin embargo habrán caminos que saldrán del estado $0$ y no volverán, ya que no hay ninguna flecha que vaya al estado $0$ desde el estado $1$ o $2$. Así notemos que la probabilidad que tiene un *random walk* de revisitar el estado $0$ es menor que $1$, al estado $0$ se le conoce como **estado transitorio**.

Supongamos ahora que queremos hacer un *random walk* desde el estado $1$, así podemos notar que después de un tiempo volvemos a visitar el estado $1$, notemos que ésto también lo cumple el estado $2$. Así la probabilidad de revisitar al estado $1$ es $1$ *(es análogo para el estado $2$)*. A estos estados se les conoce como **estados recurrentes**.

## Reducible
Decimos que una cadena de Markov es reducible sí existen en ella **estados transitorios**

Notemos que a la cadena de Markov de ésta sección podemos hacer que todos sus estados sean **recurrentes** creando una flecha del estado $2$ al estado $0$.

![Cadena de Markov con estados recurrentes](imgs/cadena_de_markov_2.png)

A ésto le llamamos que una cadena de Markov sea irreducible, cuando todos sus estados son recurrentes.

## Clases

Una clase en una cadena de Markov es un subcadena de Markov que es irreducible.

![Clases](imgs/cadena_de_markov_3.png)


# Cadenas de Markov en $n$ pasos
## ¿Cuál es la probabilidad de alcanzar a un estado $j$ desde un estado $i$ en exactamente $n$ pasos?
Así denotamos a $P_{ij}(n)$ como la probabilidad antes mencionada.
## Ejemplo 
¿Cuál es la probabilidad de alcanzar el estado **Bull Market** desde el estado **Stagnant market** en exactamente $1$ paso?
### Respuesta
$0.25$
¿Cuál es la probabilidad de alcanzar el estado **Bull Market** desde el estado **Stagnant market** en exactamente $2$ pasos?
### Respuesta
* **Stagnant Market** a **Bull Market** y de **Bull Market** a **Bull Market** esto es igual a $0.25 \times 0.9$
* **Stagnant Market** a **Stagnant Market** y de **Stagnant Market** a **Bull Market** esto es igual a $0.5 \times 0.25$
* **Stagnant Market** a **Bear Market** y de **Bear Market** a **Bull Market** esto es igual a $0.25 \times 0.15$
* **Total:** $0.25 \times 0.9 + 0.5 \times 0.25 + 0.25 \times 0.15 = 0.3875$

Notemos que lo anterior es:
$A_{20} \times A_{00} + A_{22} \times A_{21} + A_{21} \times A_{10}$
Que es igual a:
$$
\begin{bmatrix}
    A_{20} & A_{21} & A_{22}
\end{bmatrix}
\times
\begin{bmatrix}
    A_{00} \\
    A_{10} \\
    A_{20}
\end{bmatrix}
$$
De manera general tenemos que:
$$
    P_{ij}(n) = A^{n}_{ij}
$$

In [4]:
print("Probabilidad para 2 pasos: " +  str(np.linalg.matrix_power(matrix_transition, 2)[0,2]))

Probabilidad para 2 pasos: 0.03875000000000001


In [5]:
def probability_of_n_steps(matrix_transition, n, i, j):
    return np.linalg.matrix_power(matrix_transition, n)[i, j]

print("Probabilidad para los primeros 15 pasos iniciando desde Stagnant market y terminando en Bull market: ")
for i in range(1, 16):
    print("Probabilidad para " + str(i) + " pasos: " + str(probability_of_n_steps(matrix_transition, i, 0, 2)))

Probabilidad para los primeros 15 pasos iniciando desde Stagnant market y terminando en Bull market: 
Probabilidad para 1 pasos: 0.025
Probabilidad para 2 pasos: 0.03875000000000001
Probabilidad para 3 pasos: 0.04675000000000001
Probabilidad para 4 pasos: 0.05167500000000001
Probabilidad para 5 pasos: 0.05486500000000002
Probabilidad para 6 pasos: 0.057018500000000014
Probabilidad para 7 pasos: 0.05851810000000002
Probabilidad para 8 pasos: 0.059585430000000016
Probabilidad para 9 pasos: 0.06035636200000002
Probabilidad para 10 pasos: 0.06091858820000002
Probabilidad para 11 pasos: 0.06133114276000002
Probabilidad para 12 pasos: 0.06163505132400002
Probabilidad para 13 pasos: 0.06185947305040002
Probabilidad para 14 pasos: 0.06202545021032003
Probabilidad para 15 pasos: 0.062148319415248024


Lo anterior es posible gracias al **Teorema Chapman-Kolmogorov** que establece lo siguiente:
$$
    P_{ij}(n) = \sum_{k} P_{ik}(r) \times P_{kj}(n-r)
$$

In [6]:
def limit_n_to_infty(transition_matrix):
    init = transition_matrix
    i = 2
    transition_matrix = np.linalg.matrix_power(transition_matrix, i)
    while (transition_matrix != init).any():
        init = transition_matrix
        i += 1
        transition_matrix = np.linalg.matrix_power(init, i)
    return transition_matrix

# Aplicación de la cadenas de Markov: Generación de texto.
Las cadenas de Markov nos ayudan a creer modelos de lenguaje.
Para tener una cadena de Markov necesitamos lo siguiente:
* Un conjunto de estados.
* Transiciones entre éstos estados.

Para generar texto vamos a hacer que las palabras sean estados.
Supongamos que tenemos dos palabras $i$ y $j$ para calcular la probabilidad $P_{ij}$ lo único que necesitamos realizar es buscar en nuestro texto la probabilidad de que la siguiente palabra sea $j$ dado que la palabra actual es $i$ esto es:
$$
    P_{ij} = P\left( n + 1 = j | n = i \right)
$$
Sí no existe que la palabra $j$ está después que la palabra $i$ entonces la probabilidad será $0$
## Ejemplo

*My name is Sherlock Holmes. It is my business to know what other people do not know.*

![Ejemplo](imgs/NPL1.png)

El número de cada flecha es el número de veces que una palabra aparece previamente a la que apunta.

Para convertir en probabilidades lo anterior entonces por cada flecha que sale de una palabra lo dividiremos entre el número total de flechas que sale de ésta palabra.

![Ejemplo](imgs/NPL2.png)

## ¿Cómo vamos a generar texto nuevo?

Seguiremos las probabilidades dadas por la matriz de transición para ir de un estado inicial a un estado final y vamos a repetir éste proceso tanto como queramos.

# Modelos Ocultos de Markov
![Ejemplo de Modelo Oculto de Markov](imgs/HMM1.png)

In [22]:
import itertools

transition_matrix = np.matrix([[0.5, 0.3, 0.2],  # Rainy 
                               [0.4, 0.2, 0.4],  # Cloudy
                               [0, 0.3, 0.7]])   # Sunny
emission_matrix = np.matrix([[0.9, 0.1], # Sad # Happy
                             [0.6, 0.4],
                             [0.2, 0.8]])

distribution_of_markov_chain = find_distribution(transition_matrix, np.array([1, 0, 0]))
print("Distribución de probabilidad de la cadena de Markov asociada: ")
print(distribution_of_markov_chain)
# La secuencia de estados será:
# 0 -> Rainy
# 1 -> Cloudy
# 2 -> Sunny

# Secuencia observada (en columnas):
# 0 -> Sad
# 1 -> Happy
# observed_sequence = [2, 1, 2]

def compute_probability(observed_sequence, state_sequence):
    l1 = len(observed_sequence)
    l2 = len(state_sequence)
    if ((l1 < 1 or l2 < 1) or (l1 != l2)):
        raise Exception()
    p = distribution_of_markov_chain[0, state_sequence[0]]
    for i in range(1, l2):
        p *= transition_matrix[state_sequence[i - 1], state_sequence[i]]
    for i in range(l1):
        p *= emission_matrix[state_sequence[i], observed_sequence[i]]
    return p

"""
    Calcularemos la siguiente probabilidad:
    Día 1: Estuvo soleado y fuimos felices.
    Día 2: Estuvo nublado y fuimos felices.
    Día 3: Estuvo soleado y fuimos tristes.
    Ésto se traducte en:
    state_sequence = [2, 1, 2]
    observed_sequence = [1, 1, 0]
"""
state_sequence = [2, 1, 2]
observed_sequence = [1, 1, 0]
print(compute_probability(observed_sequence, state_sequence))
# ¿Cuál es la secuencia más probable de clima para los estados de ánimo dados?
def compute_maximum_probability(unit_state_sequence, observed_sequence):
    n = len(observed_sequence)
    maximum_probability_of_sequence = 0
    for sequence in itertools.product(unit_state_sequence, repeat=n):
        maximum_probability_of_sequence = max(maximum_probability_of_sequence, compute_probability(observed_sequence, sequence))
    return maximum_probability_of_sequence

print("Secuencia más probable de clima para la secuencia de estados de ánimo feliz, feliz, triste.")
compute_maximum_probability([0, 1, 2], observed_sequence)

Distribución de probabilidad de la cadena de Markov asociada: 
[[0.21818182 0.27272727 0.50909091]]
0.003909818181818177
Secuencia más probable de clima para la secuencia de estados de ánimo feliz, feliz, triste.


0.04105309090909085